In [17]:
#force Per thrusters in microNewtons
# distance in meters

def calcTorqueElectrosprayThrusters(forcePerThruster, distance, numberThrustersInArray):
    return forcePerThruster*(10**-6)*distance*numberThrustersInArray

calcTorqueElectrosprayThrusters(5, 32.5, 9)

0.0014624999999999998